In [2]:
import os
import cv2
import numpy as np
from mtcnn import MTCNN
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Initialize MTCNN detector
detector = MTCNN()

# # Function to extract features from a single image
# def extract_features(image_path):
#     image = cv2.imread(image_path)
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#     # Detect faces and landmarks
#     result = detector.detect_faces(image_rgb)
    
#     if result:  # Check if a face is detected
#         keypoints = result[0]['keypoints']  # Get keypoints of the first detected face
        
#         left_eye = keypoints['left_eye']
#         right_eye = keypoints['right_eye']
#         nose = keypoints['nose']
#         mouth_left = keypoints['mouth_left']
#         mouth_right = keypoints['mouth_right']

#         # Example features: distances between eyes, eyes and nose, mouth corners
#         eye_distance = np.linalg.norm(np.array(left_eye) - np.array(right_eye))
#         eye_nose_distance = np.linalg.norm(np.array(left_eye) - np.array(nose))
#         mouth_width = np.linalg.norm(np.array(mouth_left) - np.array(mouth_right))
        
#         return [eye_distance, eye_nose_distance, mouth_width]
#     else:
#         return None

# Function to extract features from a single image
def extract_features(image_path):
    # Attempt to read the image
    image = cv2.imread(image_path)
    
    # Check if the image was successfully loaded
    if image is None:
        print(f"Skipping image: {image_path} (Unable to read or empty image)")
        return None
    
    # Convert the image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect faces and landmarks
    result = detector.detect_faces(image_rgb)
    
    if result:  # Check if a face is detected
        keypoints = result[0]['keypoints']  # Get keypoints of the first detected face
        
        left_eye = keypoints['left_eye']
        right_eye = keypoints['right_eye']
        nose = keypoints['nose']
        mouth_left = keypoints['mouth_left']
        mouth_right = keypoints['mouth_right']

        # Example features: distances between eyes, eyes and nose, mouth corners
        eye_distance = np.linalg.norm(np.array(left_eye) - np.array(right_eye))
        eye_nose_distance = np.linalg.norm(np.array(left_eye) - np.array(nose))
        mouth_width = np.linalg.norm(np.array(mouth_left) - np.array(mouth_right))
        
        return [eye_distance, eye_nose_distance, mouth_width]
    else:
        print(f"Skipping image: {image_path} (No face detected)")
        return None

# Initialize lists to store features and labels
features = []
labels = []



# Paths to dataset
base_path = "/Facial_Paralysis_Detector/Datasets/Preprocessed_Train/"  # Replace with your dataset path
paralysis_path = os.path.join(base_path, "Stroke")


no_paralysis_folders = [
    os.path.join(base_path, "noStroke_1"),
    os.path.join(base_path, "noStroke_2"),
    os.path.join(base_path, "noParalysis_6"),
    os.path.join(base_path, "noParalysis_4"),
    os.path.join(base_path, "noStroke_3"),
    os.path.join(base_path,  "noParalysis_5"),
    os.path.join(base_path, "noParalysis_7")]


for folder in no_paralysis_folders:
    for img_file in os.listdir(folder):
        img_path = os.path.join(folder, img_file)
        feature = extract_features(img_path)
        
        if feature is not None:
            features.append(feature)
            labels.append(0)  # Label 0 for no paralysis

# Add other data (e.g., 'paralysis') similarly
paralysis_path = os.path.join(base_path, "Stroke")
for img_file in os.listdir(paralysis_path):
    img_path = os.path.join(paralysis_path, img_file)
    feature = extract_features(img_path)
    
    if feature is not None:
        features.append(feature)
        labels.append(1)  # Label 1 for paralysis

# Convert features and labels to NumPy arrays
features = np.array(features)
labels = np.array(labels)

# Split the entire dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)

# Train a classifier (Support Vector Classifier)
classifier = SVC(kernel='linear', class_weight='balanced')
classifier.fit(X_train, y_train)

# Test the classifier
y_pred = classifier.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


Skipping image: /Users/majiajia/Desktop/thisisjia/Facial_Paralysis_Detector/Datasets/Preprocessed_Train/noStroke_1/003.jpg (No face detected)
Skipping image: /Users/majiajia/Desktop/thisisjia/Facial_Paralysis_Detector/Datasets/Preprocessed_Train/noStroke_1/6a.jpg (No face detected)
Skipping image: /Users/majiajia/Desktop/thisisjia/Facial_Paralysis_Detector/Datasets/Preprocessed_Train/noStroke_1/7.jpg (No face detected)
Skipping image: /Users/majiajia/Desktop/thisisjia/Facial_Paralysis_Detector/Datasets/Preprocessed_Train/noStroke_1/6.jpg (No face detected)
Skipping image: /Users/majiajia/Desktop/thisisjia/Facial_Paralysis_Detector/Datasets/Preprocessed_Train/noStroke_1/2.jpg (No face detected)
Skipping image: /Users/majiajia/Desktop/thisisjia/Facial_Paralysis_Detector/Datasets/Preprocessed_Train/noStroke_2/14.jpg (No face detected)
Skipping image: /Users/majiajia/Desktop/thisisjia/Facial_Paralysis_Detector/Datasets/Preprocessed_Train/noStroke_2/15.jpg (No face detected)
Skipping image:

In [3]:
def extract_features_from_folder(folder_path):
    features = []
    for img_file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_file)
        if img_file.endswith(('.png', '.jpg', '.jpeg')):  # Ensure valid image files
            feature = extract_features(img_path)
            if feature is not None:
                features.append(feature)
    return np.array(features)

In [7]:
# Function to extract features from all images in a folder (already defined)

# Paths to test folders (replace with your actual paths)
paralysis_test_folder = "/Facial_Paralysis_Detector/Datasets/Preprocessed_Test/Stroke"  # Paralysis folder
no_paralysis_test_folder = "/Facial_Paralysis_Detector/Datasets/Preprocessed_Test/noStroke"  # No paralysis folder

# Extract features for both test folders
paralysis_test_features = extract_features_from_folder(paralysis_test_folder)
no_paralysis_test_features = extract_features_from_folder(no_paralysis_test_folder)

# Assign labels: 1 for paralysis, 0 for no paralysis
paralysis_test_labels = [1 for _ in range(len(paralysis_test_features))]
no_paralysis_test_labels = [0 for _ in range(len(no_paralysis_test_features))]

# Combine the features and labels from both folders
test_features = np.concatenate([paralysis_test_features, no_paralysis_test_features], axis=0)
test_labels = np.concatenate([paralysis_test_labels, no_paralysis_test_labels], axis=0)

# Now `test_features` contains all the features from the test set
# `test_labels` contains the corresponding labels (1 for paralysis, 0 for no paralysis)

# Use the trained classifier to make predictions
y_test_pred = classifier.predict(test_features)

# Evaluate the predictions on the test set
print(classification_report(test_labels, y_test_pred))


Skipping image: /Users/majiajia/Desktop/thisisjia/Facial_Paralysis_Detector/Datasets/Preprocessed_Test/Stroke/images - 2019-08-23T131842.518.jpg (No face detected)
Skipping image: /Users/majiajia/Desktop/thisisjia/Facial_Paralysis_Detector/Datasets/Preprocessed_Test/Stroke/images - 2019-08-23T125823.635.jpg (No face detected)
Skipping image: /Users/majiajia/Desktop/thisisjia/Facial_Paralysis_Detector/Datasets/Preprocessed_Test/Stroke/a-39-215x300.jpg (No face detected)
Skipping image: /Users/majiajia/Desktop/thisisjia/Facial_Paralysis_Detector/Datasets/Preprocessed_Test/Stroke/images (52).jpg (No face detected)
Skipping image: /Users/majiajia/Desktop/thisisjia/Facial_Paralysis_Detector/Datasets/Preprocessed_Test/Stroke/images (1).jpg (No face detected)
Skipping image: /Users/majiajia/Desktop/thisisjia/Facial_Paralysis_Detector/Datasets/Preprocessed_Test/Stroke/images (13).jpg (No face detected)
Skipping image: /Users/majiajia/Desktop/thisisjia/Facial_Paralysis_Detector/Datasets/Preproc